In [49]:
import pandas as pd
from airtable import airtable

import dotenv
import os
dotenv.load_dotenv()
at_api = os.environ["AIRTABLE_API_TOKEN"]
at_base = os.environ["AIRTABLE_DEVREL_BASE_ID"]

columns = ['Name','Block Explorer','Announcement Date', 'Mainnet Launch Date'
           ,'L1 Standard Bridge Contract', 'L1 NFT Bridge Contract', 'L1 BatchInbox "to address"', 'L1 BatchInbox "from address"']

In [50]:
at = airtable.Airtable(at_base, at_api)
data = at.get('OP Products')
df = pd.json_normalize(data, record_path='records')
# Rename all columns that start with 'fields.'
df.rename(columns=lambda x: x.replace('fields.', ''), inplace=True)
df = df[columns]

In [51]:
# Filter the DataFrame
filtered_df = df[df['L1 Standard Bridge Contract'].notnull() & df['L1 Standard Bridge Contract'].str.startswith('0x')]
filtered_df.reset_index(inplace=True, drop=True)
display(filtered_df)

,Name,Block Explorer,Announcement Date,Mainnet Launch Date,L1 Standard Bridge Contract,L1 NFT Bridge Contract,"L1 BatchInbox ""to address""","L1 BatchInbox ""from address"""
0,Aevo,https://explorer.aevo.xyz/,2022-09-28,2023-04-07,0x4082C9647c098a6493fb499EaE63b5ce3259c574,0x2e7d7B188D663F38c2E4fF9C59328458A2D676F0,0x253887577420Cb7e7418cD4d50147743c8041b28,0x889e21d7BA3d6dD62e75d4980A4Ad1349c61599d
